In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [5]:
tf.reset_default_graph()

samples = 1000
times = [1e-2*i for i in range(samples+1)]
sin = np.sin(times[:-1])
sin_next = np.sin(times[1:])

time_step = 10
sin = np.reshape(sin, [-1, time_step, 1])
sin_next = np.reshape(sin_next, [-1, 1])

signal = tf.placeholder(tf.float32, 
                        shape=[None, time_step, 1])
signal_next = tf.placeholder(tf.float32,
                            [None, 1])

state_size = 30
# rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=state_size)
f_rnn_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=state_size)
b_rnn_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=state_size)
# rnn_cell = tf.nn.rnn_cell.GRUCell(num_units=state_size)

(f_outputs, b_outputs), (f_state, b_state) = tf.nn.bidirectional_dynamic_rnn(f_rnn_cell, 
                                b_rnn_cell,
                                signal,
                                dtype=tf.float32)

states = tf.stack((f_outputs, b_outputs), axis=1)
reshaped_states = tf.reshape(states, [-1, state_size])
print(reshaped_states.shape)

output = tf.layers.dense(reshaped_states, 1, use_bias=False)
print(output.shape)

loss = tf.losses.mean_squared_error(signal_next, output)
train_op = tf.train.GradientDescentOptimizer(1e-2).minimize(loss)

accuracy = tf.contrib.metrics.streaming_pearson_correlation(output, signal_next)


(?, 30)
(?, 1)


In [6]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    for i in range(3000):
        _, _loss, _acc = sess.run([train_op, loss, accuracy], 
                            feed_dict={signal: sin, signal_next: sin_next})
        if i%100 == 0:
            print('step: {}, loss: {}, acc: {}'.format(i, _loss, _acc[0]))
    
    _pred = sess.run(output, feed_dict={signal: sin})
    plt.figure(1)
    plt.plot(sin_next)
    plt.figure(2)
    plt.plot(_pred)
    plt.show()

InvalidArgumentError: Incompatible shapes: [2000,1] vs. [1000,1]
	 [[Node: gradients/mean_squared_error/SquaredDifference_grad/BroadcastGradientArgs = BroadcastGradientArgs[T=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](gradients/mean_squared_error/SquaredDifference_grad/Shape, gradients/mean_squared_error/SquaredDifference_grad/Shape_1)]]

Caused by op 'gradients/mean_squared_error/SquaredDifference_grad/BroadcastGradientArgs', defined at:
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py", line 170, in _run_module_as_main
    "__main__", mod_spec)
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/traitlets/co